## Part 1: Preprocessing

In [46]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [47]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [48]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [49]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']
present_columns = [col for col in X_columns if col in attrition_df.columns]

print(f"Present columns: {present_columns}")

# Create X_df using your selected columns
X_df = attrition_df[present_columns]

# Show the data types for X_df
print(X_df.dtypes)


Present columns: ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance', 'YearsAtCompany', 'YearsSinceLastPromotion', 'NumCompaniesWorked']
Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [50]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [51]:
# Convert your X data to numeric data types however you see fit
X_train['OverTime'] = X_train['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
X_test['OverTime'] = X_test['OverTime'].apply(lambda over_time: 1 if over_time == 'Yes' else 0)
# Add new code cells as necessary

X_train['OverTime'].value_counts()

OverTime
0    781
1    321
Name: count, dtype: int64

In [52]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [53]:
# Create a OneHotEncoder for the Department column
ohe_department = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_department.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department = ohe_department.transform(y_train[['Department']])
y_test_department = ohe_department.transform(y_test[['Department']])

y_train_department

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [54]:
# Create a OneHotEncoder for the Attrition column
ohe_attrition = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
ohe_attrition.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition = ohe_attrition.transform(y_train[['Attrition']])
y_test_attrition = ohe_attrition.transform(y_test[['Attrition']])

y_train_attrition


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [55]:
# Find the number of columns in the X training data
number_of_columns = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(number_of_columns,))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared_layer1')(input_layer)
shared_layer2 = layers.Dense(64, activation='relu', name='shared_layer2')(shared_layer1)

In [56]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
hidden_deparrement = layers.Dense(64, activation='relu', name='hidden_department')(shared_layer2)

# Create the output layer
output_department = layers.Dense(y_train_department.shape[1], activation='softmax', name='output_department')(hidden_deparrement)


In [57]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
hidden_attrition = layers.Dense(32, activation='relu', name='hidden_attrition')(shared_layer2)

# Create the output layer
output_attrition = layers.Dense(y_train_attrition.shape[1], activation='sigmoid', name='output_attrition')(hidden_attrition)


In [63]:
# Create the model
model = Model(inputs=[input_layer], outputs=[output_department, output_attrition])

# Compile the model with the correct loss and metrics
model.compile(optimizer='adam', 
              loss={'output_department': 'categorical_crossentropy', 
                    'output_attrition': 'binary_crossentropy'},
              metrics={'output_department': 'accuracy',
                       'output_attrition': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 64)        │        704 │ input_layer_2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 64)        │      4,160 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_department   │ (None, 64)        │      4,160 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hidden_attrition    │ (None, 32)        │      2,080 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_department   │ (None, 3)         │        195 │ hidden_departmen… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_attrition    │ (None, 2)         │         66 │ hidden_attrition… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,365 (44.39 KB)

 Trainable params: 11,365 (44.39 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
# Train the model
history = model.fit(
    X_train_scaled, 
    {'output_department': y_train_department, 'output_attrition': y_train_attrition},
    epochs=100,
    batch_size=32,
    verbose=1
)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - loss: 1.5121 - output_attrition_accuracy: 0.8027 - output_department_accuracy: 0.6056 
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step - loss: 1.2852 - output_attrition_accuracy: 0.8335 - output_department_accuracy: 0.6249
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2077 - output_attrition_accuracy: 0.8348 - output_department_accuracy: 0.6663 
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - loss: 1.1111 - output_attrition_accuracy: 0.8710 - output_department_accuracy: 0.6513
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - loss: 1.1561 - output_attrition_accuracy: 0.8464 - output_department_accuracy: 0.6313
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 635us/step - loss: 1.1171 - output_attrition_accuracy: 0.8562 - output_department_accuracy: 0.6423
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 1.1060 - output_attrition_accuracy: 0.8630 - output_department_accuracy: 0.6290
Epoch 8/100
3

In [65]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'output_department': y_test_department, 'output_attrition': y_test_attrition}, verbose=1)

test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step - loss: 3.4850 - output_attrition_accuracy: 0.8659 - output_department_accuracy: 0.5492


[3.6640608310699463, 0.83152174949646, 0.570652186870575]

In [66]:
# Print the accuracy for both department and attrition
print(f"Department prediction accuracy: {test_results[1]}")
print(f"Attrition prediction accuracy: {test_results[2]}")

Department prediction accuracy: 0.83152174949646
Attrition prediction accuracy: 0.570652186870575


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy is not always the best metric for evaluating a model’s performance. In scenarios where the majority of employees remain and only a few depart, a model that predicts everyone will stay could achieve high accuracy. 
2. To predict whether an employee will leave, I utilized the sigmoid function. This function outputs a value between 0 and 1, making it ideal for binary classification tasks. For determining the best department for an employee, I employed the softmax function. This function generates probabilities for each department.
3. Balance the Data: If there’s an imbalance in the number of employees across different departments, balance the dataset by either oversampling the smaller departments or generating synthetic data to even out the distribution.

Test More Thoroughly: Use cross-validation by training and testing the model on different subsets of the data multiple times to obtain a more reliable assessment of its performance.
